In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

def polynomial_kernel(x1, x2, degree=3, coef0=1):
    return (np.dot(x1, x2.T) + coef0) ** degree

class KernelSVM:
    def __init__(self, lr=0.001, lambda_param=0.01, n_iters=100, degree=3, coef0=1):
        self.lr = lr
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.degree = degree
        self.coef0 = coef0
        self.alpha = None
        self.b = 0
        self.X_train = None
        self.y_train = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.X_train = X
        self.y_train = y
        self.alpha = np.zeros(n_samples)
        self.b = 0
        
        K = polynomial_kernel(X, X, self.degree, self.coef0)
        
        for _ in range(self.n_iters):
            for i in range(n_samples):
                pred = np.sum(self.alpha * self.y_train * K[:, i]) + self.b
                condition = y[i] * pred >= 1
                
                if condition:
                    self.alpha[i] -= self.lr * (self.lambda_param * self.alpha[i])
                else:
                    self.alpha[i] += self.lr * (1 - y[i] * pred)
                    self.b += self.lr * y[i]

    def predict(self, X_test):
        K = polynomial_kernel(self.X_train, X_test, self.degree, self.coef0)
        decision = np.dot(self.alpha * self.y_train, K) + self.b
        return np.sign(decision)

try:
    data = pd.read_csv(r"C:\Users\ROHIT\OneDrive\Desktop\SM\AIML\datasets\student_performance_dataset_20.csv")
    
    feature_columns = ['Study_Hours_per_Week', 'Attendance_Rate', 'Internal_Scores']
    target_column = 'Pass_Fail'
    
    X = data[feature_columns].values
    y = np.where(data[target_column] == 'Fail', -1, 1)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    print("\nTraining SVM (Polynomial Kernel, degree=2) from Scratch...")
    svm_model_20 = KernelSVM(lr=0.001, lambda_param=0.01, n_iters=100, degree=2, coef0=1)
    
    svm_model_20.fit(X_train, y_train)
    
    y_pred = svm_model_20.predict(X_test)
    
    print("\n--- Evaluation (Problem 20) ---")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    
    print("\nClassification Report (Precision, Recall, F1-Score):")
    print(classification_report(y_test, y_pred, target_names=['Fail (-1)', 'Pass (1)']))

except FileNotFoundError:
    print("Error: 'student_performance_dataset_20.csv' not found.")
except KeyError:
    print("Error: CSV must contain 'Study_Hours_per_Week', 'Attendance_Rate', 'Internal_Scores', and 'Pass_Fail' columns.")
except Exception as e:
    print(f"An error occurred: {e}")

Good afternoon, ma’am. This program builds a from-scratch Support Vector Machine (SVM) classifier with a polynomial kernel to predict whether a student will Pass or Fail based on study hours per week, attendance rate, and internal scores. It loads the dataset, preprocesses features, trains our custom kernel SVM using a simple gradient-style update on the dual variables, and finally evaluates performance with a confusion matrix and a classification report.
First, I import the libraries: NumPy and pandas for numerical arrays and data frames, scikit-learn’s train_test_split to create train/test sets, StandardScaler to standardize features, and the metrics classification_report and confusion_matrix for evaluation.
I define a polynomial kernel function polynomial_kernel(x1, x2, degree=3, coef0=1) that returns (x1⋅x2⊤+coef0)degree(x_1 \cdot x_2^\top + \text{coef0})^{\text{degree}}(x1​⋅x2⊤​+coef0)degree. This lets the SVM separate classes in a higher-dimensional feature space without explicitly creating polynomial features (the “kernel trick”).
Next is the custom KernelSVM class. In __init__, I store hyperparameters: learning rate lr, regularization lambda_param, number of training passes n_iters, and the kernel’s degree and coef0. I also prepare placeholders: the dual weights alpha (one per training point), the bias b, and references to X_train and y_train so I can reuse them at prediction time.
In fit(self, X, y), I save the training data and initialize alpha to zeros and b to 0. I precompute the Gram matrix K = polynomial_kernel(X, X, degree, coef0), where each entry K[j, i] is the kernel similarity between sample j and sample i. Then I run the main training loop for n_iters passes; inside each pass I iterate over all training samples. For sample i, I compute the current decision value
pred = Σ_j (alpha[j] * y[j] * K[j, i]) + b.
I check the margin condition y[i] * pred >= 1. If it’s satisfied (the point is correctly classified with margin), I apply only regularization by shrinking alpha[i] a little: alpha[i] -= lr * (lambda_param * alpha[i]). If the condition is violated (point is misclassified or inside the margin), I push the model to correct it: alpha[i] += lr * (1 − y[i] * pred) and also adjust the bias b += lr * y[i]. Repeating these small updates moves the separating boundary toward a configuration that maximizes the margin while penalizing violations—this is a simple gradient-style approximation to the SVM’s hinge-loss optimization in the dual.
The predict(self, X_test) method computes the kernel between all training points and each test point: K = polynomial_kernel(self.X_train, X_test, ...). It then evaluates the decision function
decision = (alpha * y_train) · K + b
and returns np.sign(decision) as −1 for Fail and +1 for Pass.
In the try block I load the student dataset from the given CSV path. I choose three input columns—Study_Hours_per_Week, Attendance_Rate, and Internal_Scores—as features, and Pass_Fail as the target. I convert the labels to −1 for “Fail” and +1 for “Pass”, which matches the SVM’s expected label format. I split into training (80%) and testing (20%) with stratification so the Pass/Fail ratio is preserved. Because SVMs are sensitive to feature scales, I standardize inputs using StandardScaler: I fit it on the training set and transform both train and test (this prevents data leakage).
I then announce training an SVM with a polynomial kernel of degree 2 by creating KernelSVM(lr=0.001, lambda_param=0.01, n_iters=100, degree=2, coef0=1). I call fit on the scaled training data and then predict on the scaled test data to obtain y_pred. For evaluation, I print the confusion matrix—which shows true/false Pass/Fail counts—and a detailed classification report with precision, recall, and F1-score for each class, labeled as “Fail (−1)” and “Pass (1)”. If needed, I also handle common errors: a missing file or missing columns will trigger clear messages in the except blocks.
In summary, the script: loads and cleans the student performance data; encodes the target to −1/+1; standardizes features; trains a from-scratch kernel SVM with a degree-2 polynomial kernel by iteratively updating dual weights and bias using a hinge-loss condition; and evaluates the classifier with standard metrics to show how well it distinguishes Pass vs Fail based on study hours, attendance, and internal marks.